# phenotypeGivenGenotypeMendelianFactor

In [23]:
cd ../assignment2

In [43]:
isDominant = 0;
genotypeVar = 1;
phenotypeVar = 3;

In [44]:
phenotypeFactor = struct('var', [], 'card', [], 'val', []);


In [47]:
phenotypeFactor.var = [phenotypeVar genotypeVar];
phenotypeFactor.card = [2 3];
if isDominant,
    phenotypeFactor.val = [1 0 1 0 0 1];
else
    phenotypeFactor.val = [0 1 0 1 1 0];
end

P(phenotype | genotype)


P(phenotype=True | FF)

P(phenotype=False | FF)

P(phenotype=True | Ff)

P(phenotype=False | Ff)

P(phenotype=True | ff)

P(phenotype=False | ff)


In [48]:
phenotypeFactor

phenotypeFactor =

  scalar structure containing the fields:

    var =

       3   1

    card =

       2   3

    val =

       0   1   0   1   1   0




# phenotypeGivenGenotypeFactor

In [65]:
phenotypeFactor = struct('var', [], 'card', [], 'val', []);


In [66]:
alphaList = [0.8; 0.6; 0.1];
genotypeVar = 1;
phenotypeVar = 3;

In [67]:
phenotypeFactor.var = [phenotypeVar genotypeVar];
phenotypeFactor.card = [2 length(alphaList)];

In [68]:
val = [];
for i=1:length(alphaList),
    prob = alphaList(i);
    val = [val [prob 1-prob]];
end
phenotypeFactor.val = val;

In [69]:
phenotypeFactor

phenotypeFactor =

  scalar structure containing the fields:

    var =

       3   1

    card =

       2   3

    val =

       0.8000   0.2000   0.6000   0.4000   0.1000   0.9000




# genotypeGivenAlleleFreqsFactor

In [70]:
alleleFreqs = [0.1; 0.9];
genotypeVar = 1;

In [71]:
numAlleles = length(alleleFreqs);


In [90]:
genotypeFactor = struct('var', [], 'card', [], 'val', []);


In [72]:
[allelesToGenotypes, genotypesToAlleles] = generateAlleleGenotypeMappers(numAlleles);


In [112]:
genotypeFactor.var = [genotypeVar];
genotypeFactor.card = [numAlleles+1];
for i=1:length(genotypesToAlleles),
    idx = genotypesToAlleles(i,:); 
    prob = alleleFreqs(idx(1))*alleleFreqs(idx(2));
    if idx(1)!=idx(2),
        prob = prob * 2;
    end
    genotypeFactor = SetValueOfAssignment(genotypeFactor, i, prob);
end

In [113]:
genotypeFactor

genotypeFactor =

  scalar structure containing the fields:

    var = [](0x0)
    card = 3
    val =

       0.010000   0.180000   0.810000




# genotypeGivenParentsGenotypesFactor

In [312]:
genotypeFactor = struct('var', [], 'card', [], 'val', []);


In [370]:
numAlleles = 3;
genotypeVarChild = 3;
genotypeVarParentOne = 1;
genotypeVarParentTwo = 2;

In [371]:
[allelesToGenotypes, genotypesToAlleles] = generateAlleleGenotypeMappers(numAlleles);


In [372]:
allelesToGenotypes(1,2,1)

ans = 2


In [373]:
allelesToGenotypes

allelesToGenotypes =

   1   2   3
   2   4   5
   3   5   6



In [374]:
genotypeFactor.var = [genotypeVarChild, genotypeVarParentOne, genotypeVarParentTwo ];

In [375]:
card = numAlleles*(numAlleles-1)/2+numAlleles

card = 6


In [376]:
genotypeFactor.card = [card card card];

In [377]:
allele_comb_parent_two(2)

ans = 2


In [378]:
genotypesToAlleles

genotypesToAlleles =

   1   1
   1   2
   1   3
   2   2
   2   3
   3   3



In [379]:
val = [];
for j=1:length(genotypesToAlleles),
    for z=1:length(genotypesToAlleles),
        allele_comb_parent_one = genotypesToAlleles(z, :);
        allele_comb_parent_two = genotypesToAlleles(j, :);
        unq = [];
        for idx1=1:length(allele_comb_parent_one),
            for idx2=1:length(allele_comb_parent_two),
                unq = [unq; [allele_comb_parent_one(idx1) allele_comb_parent_two(idx2)]];
            end
        end
        unq = unique(unq, 'rows');
        unnormalized_probs = zeros(1, card);
        for idx=1:size(unq,1),
            index = unq(idx,:);
            if index(1)>index(2),
                index = [index(2) index(1)];
            end
            [_, index] = max(sum(genotypesToAlleles==index,2));
            unnormalized_probs(index) = unnormalized_probs(index)+1;
        end
        unnormalized_probs = unnormalized_probs/sum(unnormalized_probs);
        val = [val unnormalized_probs];
    end
end

In [380]:
genotypeFactor.val = val;

In [381]:
genotypeFactor

genotypeFactor =

  scalar structure containing the fields:

    var =

       3   1   2

    card =

       6   6   6

    val =

     Columns 1 through 8:

       1.0000        0        0        0        0        0   0.5000   0.5000

     Columns 9 through 16:

            0        0        0        0   0.5000        0   0.5000        0

     Columns 17 through 24:

            0        0        0   1.0000        0        0        0        0

     Columns 25 through 32:

            0   0.5000   0.5000        0        0        0        0        0

     Columns 33 through 40:

       1.0000        0        0        0   0.5000   0.5000        0        0

     Columns 41 through 48:

            0        0   0.2500   0.5000        0   0.2500        0        0

     Columns 49 through 56:

       0.2500   0.2500   0.2500        0   0.2500        0        0   0.5000

     Columns 57 through 64:

            0   0.5000        0        0        0   0.2500   0.2500   0.2500

     Columns 65 

# constructGeneticNetwork

In [382]:
pedigree = struct('parents', [0,0;1,3;0,0]);
pedigree.names = {'Ira','James','Robin'};
alleleFreqs = [0.1; 0.9];
alphaList = [0.8; 0.6; 0.1];

In [383]:
numPeople = length(pedigree.names);


In [385]:
factorList(2*numPeople) = struct('var', [], 'card', [], 'val', []);

numAlleles = length(alleleFreqs);

In [437]:
for idx=1:numPeople,
    parents = pedigree.parents(idx,:);
    if ~sum(parents)
        factorList(idx) = genotypeGivenAlleleFreqsFactor(alleleFreqs, idx);
    else
        factorList(idx) = genotypeGivenParentsGenotypesFactor(numAlleles, idx, parents(1), parents(2));
    end
    factorList(numPeople+idx) = phenotypeGivenGenotypeFactor(alphaList, idx, numPeople+idx);

end

# phenotypeGivenCopiesFactor

In [836]:
alphaListThree = [0.8; 0.6; 0.1; 0.5; 0.05; 0.01];
numAllelesThree = 3;
genotypeVarMotherCopy = 1;
genotypeVarFatherCopy = 2;
phenotypeVar = 3;

In [837]:
[allelesToGenotypes, genotypesToAlleles] = generateAlleleGenotypeMappers(numAllelesThree);


In [838]:
phenotypeFactor = struct('var', [], 'card', [], 'val', []);


In [839]:
phenotypeFactor.var = [phenotypeVar genotypeVarMotherCopy  genotypeVarFatherCopy];

In [840]:
phenotypeFactor.card = [numAllelesThree-1 numAllelesThree numAllelesThree];

In [844]:
phenotypeFactor.val = zeros(1, prod(phenotypeFactor.card));

Assignments = IndexToAssignment(1:length(phenotypeFactor.val), phenotypeFactor.card);


In [846]:
Assignments = IndexToAssignment(1:length(phenotypeFactor.val), phenotypeFactor.card);
for i=1:phenotypeFactor.card(1)
    for j=1:phenotypeFactor.card(2)
        for k=1:phenotypeFactor.card(3)
            ind = AssignmentToIndex([i,j,k], phenotypeFactor.card);
            if(i==1)
                phenotypeFactor.val(ind) = alphaList(allelesToGenotypes(j,k));
            else
                phenotypeFactor.val(ind) = 1-alphaList(allelesToGenotypes(j,k));
            end;
        end;
    end;
end;

In [847]:
size(val)

ans =

    1   18



In [848]:
val

val =

 Columns 1 through 7:

   0.800000   0.200000   0.600000   0.400000   0.100000   0.900000   0.600000

 Columns 8 through 14:

   0.400000   0.500000   0.500000   0.050000   0.950000   0.100000   0.900000

 Columns 15 through 18:

   0.050000   0.950000   0.010000   0.990000



In [528]:
val

val =

 Columns 1 through 7:

   0.800000   0.200000   0.600000   0.400000   0.100000   0.900000   0.600000

 Columns 8 through 14:

   0.400000   0.500000   0.500000   0.050000   0.950000   0.100000   0.900000

 Columns 15 through 18:

   0.050000   0.950000   0.010000   0.990000



# constructDecoupledGeneticNetwork

In [849]:
clear

In [850]:
pedigree = struct('parents', [0,0;1,3;0,0]);
pedigree.names = {'Ira','James','Robin'};
alleleFreqs = [0.1; 0.7; 0.2];
alphaList = [0.8; 0.6; 0.1; 0.5; 0.05; 0.01];

In [851]:
numPeople = length(pedigree.names);
numAlleles = length(alleleFreqs); % Number of alleles
factorList(3*numPeople) = struct('var', [], 'card', [], 'val', []);


In [852]:
pedigree.parents(1,:)

ans =

   0   0



In [853]:
pedigree.names{pedigree.parents(2,1)}

ans = Ira


In [854]:
counter = 0;
for gen_type=[1 2],
    for idx=1:numPeople,
        parent = pedigree.parents(idx,gen_type);
        if ~sum(parent)
            factorList(idx+counter) = childCopyGivenFreqsFactor(alleleFreqs, idx+counter);
        else
            factorList(idx+counter) = childCopyGivenParentalsFactor(numAlleles, idx+counter, parent, parent+numPeople);
        end
    end
    counter = numPeople;
end


In [856]:
for idx=1:numPeople,
    factorList(2*numPeople+idx) = phenotypeGivenCopiesFactor(alphaList, numAlleles, idx, numPeople+idx, 2*numPeople+idx);
end

In [857]:
sampleFactorListDecoupled = load('sampleFactorListDecoupled.mat');

In [866]:
sampleFactorListDecoupled.sampleFactorListDecoupled(9)

ans =

  scalar structure containing the fields:

    var =

       9   3   6

    card =

       2   3   3

    val =

     Columns 1 through 6:

       0.800000   0.200000   0.600000   0.400000   0.100000   0.900000

     Columns 7 through 12:

       0.600000   0.400000   0.500000   0.500000   0.050000   0.950000

     Columns 13 through 18:

       0.100000   0.900000   0.050000   0.950000   0.010000   0.990000




In [865]:
factorList(9)

ans =

  scalar structure containing the fields:

    var =

       9   3   6

    card =

       2   3   3

    val =

     Columns 1 through 6:

       0.800000   0.200000   0.600000   0.400000   0.100000   0.900000

     Columns 7 through 12:

       0.600000   0.400000   0.500000   0.500000   0.050000   0.950000

     Columns 13 through 18:

       0.100000   0.900000   0.050000   0.950000   0.010000   0.990000




# ConstructSigmoidPhenotypeFactor

In [914]:
alleleWeights = {[3, -3], [0.9, -0.8]};
geneCopyVarOneList = [1; 2];
geneCopyVarTwoList = [4; 5];
phenotypeVar = 3;

In [915]:
phenotypeFactor = struct('var', [], 'card', [], 'val', []);

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% INSERT YOUR CODE HERE
% Note that computeSigmoid.m will be useful for this function.
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  

% Fill in phenotypeFactor.var.  This should be a 1-D row vector.
% Fill in phenotypeFactor.card.  This should be a 1-D row vector.
phenotypeFactor.var = [phenotypeVar, geneCopyVarOneList', geneCopyVarTwoList'];
phenotypeFactor.card = ones(1, 5) * 2;

phenotypeFactor.val = zeros(1, prod(phenotypeFactor.card));
% Replace the zeros in phentoypeFactor.val with the correct values.

assignments = IndexToAssignment(1:length(phenotypeFactor.val), phenotypeFactor.card);
% assignments = assignments - 1;  % reset to zero index
phenotypeZero = assignments(find(assignments(:, 1)==1), 2:end);

n_alleles = length(alleleWeights{1});
n_genes = length(geneCopyVarOneList);

for pheno=1:size(phenotypeZero, 1)
    a = phenotypeZero(pheno, :);

    z = 0;
    for j=1:n_genes
        for i=1:n_alleles
            z = z + alleleWeights{j}(i) * sum(a(j:n_genes:end)==i);
        end
    end

    % z = sum(a(1:n_genes:end)) * alleleWeights{1}(1) + ...
    %     sum(a(2:n_genes:end)) * alleleWeights{1}(2) + ...
    %     sum(a(2:n_genes:end)) * alleleWeights{2}(1) + ...
    %     sum(a(1:n_genes:end)) * alleleWeights{2}(2)
    z2 = computeSigmoid(z);

    phenotypeFactor.val(pheno*n_genes - 1) = z2;
    phenotypeFactor.val(pheno*n_genes) = 1 - z2;

end

In [918]:
phenotypeFactor.val

ans =

 Columns 1 through 6:

   9.9959e-01   4.0957e-04   8.5815e-01   1.4185e-01   9.9776e-01   2.2378e-03

 Columns 7 through 12:

   5.2498e-01   4.7502e-01   8.5815e-01   1.4185e-01   1.4774e-02   9.8523e-01

 Columns 13 through 18:

   5.2498e-01   4.7502e-01   2.7320e-03   9.9727e-01   9.9776e-01   2.2378e-03

 Columns 19 through 24:

   5.2498e-01   4.7502e-01   9.8787e-01   1.2128e-02   1.6798e-01   8.3202e-01

 Columns 25 through 30:

   5.2498e-01   4.7502e-01   2.7320e-03   9.9727e-01   1.6798e-01   8.3202e-01

 Columns 31 and 32:

   5.0020e-04   9.9950e-01



In [889]:
geneCopyVarParentOneList(2)

ans = 2


In [899]:
computeSigmoid(3*(1+0)+0.9*(1+0))

ans = 0.9802
